## 080321
## This notebook contains functions useful for analyzing Omics Integrator Outputs
## Cell specific function and general functions

In [2]:
import pickle
from gprofiler import GProfiler
import numpy as np
import pandas as pd
import networkx as nx
import OmicsIntegrator as oi
import community
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

In [3]:
import os

In [4]:
os.getcwd()

'/nfs/latdata/bkang/Omics Integrator Pipeline'

In [5]:
# Plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "Arial"

import seaborn as sns

In [6]:
mynetworks = pickle.load(open("./OI_Results/pickles/080421_networks/mynetworks.pkl", "rb"))
mynetworks

{'Naive': <networkx.classes.graph.Graph at 0x7f26635075f8>,
 '1_Astrocyte': <networkx.classes.graph.Graph at 0x7f24cf7d8d68>,
 '1_GABAergic': <networkx.classes.graph.Graph at 0x7f24cf6fc7b8>,
 '1_Microglia': <networkx.classes.graph.Graph at 0x7f24cf6985c0>,
 '1_OPC': <networkx.classes.graph.Graph at 0x7f24cf61cbe0>,
 '1_Oligodendrocyte': <networkx.classes.graph.Graph at 0x7f24cf5b1e48>,
 '3_Astrocyte': <networkx.classes.graph.Graph at 0x7f24cf4c4eb8>,
 '3_GABAergic': <networkx.classes.graph.Graph at 0x7f24cf400240>,
 '3_Microglia': <networkx.classes.graph.Graph at 0x7f24cf394278>,
 '3_OPC': <networkx.classes.graph.Graph at 0x7f24cf30ccc0>,
 '3_Oligodendrocyte': <networkx.classes.graph.Graph at 0x7f24cf29afd0>,
 '5_Astrocyte': <networkx.classes.graph.Graph at 0x7f24cf1c7470>,
 '5_GABAergic': <networkx.classes.graph.Graph at 0x7f24cf157ba8>,
 '5_Microglia': <networkx.classes.graph.Graph at 0x7f24cf0f7860>,
 '5_OPC': <networkx.classes.graph.Graph at 0x7f24cf0086d8>,
 '5_Oligodendrocyte': 

In [ ]:
L1 = mynetworks['Naive']

## Useful functions for analyzing randomization output from OI

In [1]:

# functions to use
def get_avg_cost(graph):
    df = nx.to_pandas_edgelist(graph)
    return sum(df['cost'])/len(df)

def jaccard(L1, L2):
    for i in range(len(L1)):
        for j in range(len(L2)):
            if (L1[i][0] == L2[j][1]) & (L1[i][1] == L2[j][0]):
                L2[j] = (L2[j][1], L2[j][0])
            
    intersection = len(list(set(L1).intersection(L2)))
    #print(intersection)
    union = (len(L1) + len(L2)) - intersection
    return float(intersection) / union

# use this to convert greene et al network into gene symbols
def convert_entrez_to_symbols(df_input):
    df = df_input.copy()
    gene_symbols = {}
    mg = mygene.MyGeneInfo()
    genes = df[0].values.tolist()
    annotations = mg.getgenes(genes, fields='symbol,entrezgene')
    #print(annotations)
    for gene in annotations:
        if 'symbol' in gene.keys():
            gene_symbols[gene['query']] = gene['symbol']
        else:
            gene_symbols[gene['query']] = gene['query']
    
    symbol_list_to_add = []
    for gene in genes:
        symbol_list_to_add.append(gene_symbols[gene])
    df[0] = symbol_list_to_add
    
    gene_symbols = {}
    mg = mygene.MyGeneInfo()
    genes = df[1].values.tolist()
    annotations = mg.getgenes(genes, fields='symbol,entrezgene')
    #print(annotations)
    for gene in annotations:
        if 'symbol' in gene.keys():
            gene_symbols[gene['query']] = gene['symbol']
        else:
            gene_symbols[gene['query']] = gene['query']
            
    symbol_list_to_add = []
    for gene in genes:
        symbol_list_to_add.append(gene_symbols[gene])
    df[1] = symbol_list_to_add
    return df


def generate_basic_statistics(graph,robust_results): # graph.generate_basic_statistics
    robust_summary = {}

    for paramstring, robust_network in robust_results.items(): 
        
        if robust_network.number_of_nodes() != 0: 
                
            robust_df = oi.get_networkx_graph_as_dataframe_of_nodes(robust_network)
            
            robust_edges_df = nx.to_pandas_edgelist(robust_network)
           # sum_cost = 
            robust_summary[paramstring] = {
                "W":                paramstring.split("_")[1],
                "B":                paramstring.split("_")[3],
                "G":                paramstring.split("_")[5],
                "K":                paramstring.split("_")[7],
                "size":             len(robust_df), 
                "min_robustness":   robust_df.robustness.min(), 
                "mean_robustness":  robust_df.robustness.mean(), 
                "max_specificity":  robust_df.specificity.max(),
                "mean_specificity": robust_df.specificity.mean(), 
                "mean_log_degree":  np.log2(robust_df.degree).mean(), 
                "std_log_degree":   np.log2(robust_df.degree).std(), 
                "KS_statistic":     stats.ks_2samp(np.log2(robust_df.degree), np.log2(graph.node_attributes.degree))[0],
                "num_fly_prize": len(robust_df[robust_df.source == 'fly genetic screen']),
                "num_HD_prize" : len(robust_df[robust_df.source == "huntington's AOO modifiers"]),
                "mean_edge_cost": get_avg_cost(robust_network)
                
                                     
                }
                
    robust_summary = pd.DataFrame.from_dict(robust_summary, orient='index')
    return robust_summary

def filter_by_max_specificity(robust_summary, max_specificity):
    # initial filter by max-specificity > 0.9
    robust_summary = robust_summary[robust_summary['max_specificity'] >= 0.90]
    
    node_attributes_df = oi.get_networkx_graph_as_dataframe_of_nodes(nxgraph)
    node_attributes_df = node_attributes_df[node_attributes_df["robustness"] > min_robustness]
    node_attributes_df = node_attributes_df[node_attributes_df["specificity"] < max_specificity]
    robust_network = nxgraph.subgraph(node_attributes_df.index[:]).copy()
    return robust_network

def get_filtered_subgraph_from_randomizations(nxgraph, min_robustness, max_specificity):

    if nxgraph.number_of_nodes() ==0:
        oi.logger.warning("Augmented forest is empty.")
        return nxgraph

    node_attributes_df = oi.get_networkx_graph_as_dataframe_of_nodes(nxgraph)
    node_attributes_df = node_attributes_df[node_attributes_df["robustness"] > min_robustness]
    node_attributes_df = node_attributes_df[node_attributes_df["specificity"] < max_specificity]
    robust_network = nxgraph.subgraph(node_attributes_df.index[:]).copy()
    return robust_network

# Returns a dictionary containing subgraph copies of each louvain cluster
# in original dataframe, cluster is in string
def get_subgraph_from_clusters(nxgraph, cluster_type):
    clusters = {}
    if nxgraph.number_of_nodes()==0:
        oi.logger.warning("Augmented forest is empty.")
        return nxgraph
    node_attributes_df = oi.get_networkx_graph_as_dataframe_of_nodes(nxgraph)
    temp_list = node_attributes_df.loc[:,cluster_type].tolist()
    mylist = [int(i) for i in temp_list]
    for cluster in range(max(mylist)+1):
        subset = node_attributes_df[node_attributes_df[cluster_type] == str(cluster)]
        clusters[cluster] = nxgraph.subgraph(subset.index[:]).copy()

    return clusters

# lv_clusters can be replaced with greedy or leiden cluster
# lv_clusters param is a dictionary containing graphs
def get_pathway_enrichment(sourcelist, lv_clusters, prec_thresh, int_size_thresh):
    gp = GProfiler(return_dataframe=True)
    lv_clusters_df = {cluster: oi.get_networkx_graph_as_dataframe_of_nodes(network) for cluster, network in lv_clusters.items()}
    Qdict = {cluster: df.index.values.tolist() for cluster, df in lv_clusters_df.items()}
    enrichment_results = {cluster: gp.profile(organism='hsapiens',query=qlist, sources=sourcelist, no_evidences = False) for cluster, qlist in Qdict.items()}

    enrichment_results = {cluster: df[(df['precision'] >= prec_thresh) & (df['intersection_size'] >= int_size_thresh)] for cluster, df in enrichment_results.items()}
    
    return enrichment_results

def annotate_pathway(network, enrichment_result, loc_list):
    # enrichment_result is dataframe (for each cluster)
    for i in range(len(loc_list)):
        loc = loc_list[i]
        nodes = enrichment_result.iloc[loc]['intersections']
        pval = enrichment_result.iloc[loc]['p_value']
        source = enrichment_result.iloc[loc]['native']
        name = enrichment_result.iloc[loc]['name']

        for node in nodes:
            pathname = 'path_' +str(i+1)
            pvalname = 'pval_' +str(i+1)
            nx.set_node_attributes(network, {node: {pathname: source + ': ' + name}})
            nx.set_node_attributes(network, {node: {pvalname: str(pval)}})
    
def annotate_pathway_overlap(network, enrichment_result, loc_list):
    # enrichment_result is dataframe (for each cluster)
    loc_list.reverse()
    for i in range(len(loc_list)):
        loc = loc_list[i]
        nodes = enrichment_result.iloc[loc]['intersections']
        pval = enrichment_result.iloc[loc]['p_value']
        source = enrichment_result.iloc[loc]['native']
        name = enrichment_result.iloc[loc]['name']

        for node in nodes:
            nx.set_node_attributes(network, {node: {'pathway': source + ': ' + name}})
            
            
def invert(list_of_lists): return {item: i for i, list in enumerate(list_of_lists) for item in list}

def greedy_clustering(nxgraph):
    
    clustering = pd.Series(invert(nx.algorithms.community.greedy_modularity_communities(nxgraph)), 
                           name='greedy_clusters').astype(str).reindex(nxgraph.nodes())
    nx.set_node_attributes(nxgraph, clustering.to_frame().to_dict(orient='index'))
    
def leiden_clustering(nxgraph):
    
    clustering = pd.Series(invert(algorithms.leiden(nxgraph).communities), name='leiden_clusters').astype(str).reindex(nxgraph.nodes())
    nx.set_node_attributes(nxgraph, clustering.to_frame().to_dict(orient='index'))
    
def louvain_clustering(nxgraph,resolution, random_state):
    nx.set_node_attributes(nxgraph, {node: {'louvain_clusters':str(cluster)} for node,cluster in community.best_partition(nxgraph, resolution=resolution, 
                                                                                                                          random_state=random_state).items()})
    
def sort_graph(nxgraph):
    nodes_sorted = sorted(nxgraph.nodes(data=True), key= lambda x: (x[1]['source']))
    edges = sorted(nxgraph.edges(data=True))
    sorted_graph = nx.Graph()
    sorted_graph.add_nodes_from(nodes_sorted)
    sorted_graph.add_edges_from(edges)
    return sorted_graph

def get_avg_degree(nxgraph):
    s = 0
    for node in nxgraph.nodes():
        s += nxgraph.degree(node)
    return s/nxgraph.number_of_nodes()

def get_network_stats(clusters):
    num_nodes = []
    avg_degree = []
    
    for cluster, network in clusters.items():
        print(cluster, network.number_of_nodes(), get_avg_degree(network))



In [7]:
def get_robust_subgraph_from_randomizations(nxgraph, max_size=400, min_component_size=5, min_robustness=0):
    """
    Given a graph with robustness attributes, take the top `max_size` robust nodes and
    prune any "small" components.

    Arguments:
        nxgraph (networkx.Graph): Network from randomization experiment
        max_size (int): Max size of robust network

    Returns:
        networkx.Graph: Robust network
    """

    # TODO: Potential alternative approach - from entire network, attempt to remove lowest robustness node.
    # If removal results in a component of size less than min_size, do not remove.

    if nxgraph.number_of_nodes() == 0:
   #     logger.warning("Augmented forest is empty.")
        return nxgraph

    # Get indices of top nodes sorted by high robustness, then low specificity. Don't allow nodes with robustness = 0.
    node_attributes_df = oi.get_networkx_graph_as_dataframe_of_nodes(nxgraph)
    node_attributes_df = node_attributes_df[node_attributes_df["robustness"] > min_robustness]
    node_attributes_df.sort_values(["robustness", "specificity"], ascending=[False, True], inplace=True)

    # Find the largest subgraph of `nxgraph` such that the number of nodes, after filtering out small components, is 
    # not more than `max_size`. This is accomplished by progressively loosening the threshold for inclusion in the subgraph. 
    for top_n in np.arange(min(max_size, len(node_attributes_df)), len(node_attributes_df)+1): 
        # Get robust subnetwork and remove small components.
        robust_network = nxgraph.subgraph(node_attributes_df.index[:top_n])
        robust_network = filter_graph_by_component_size(robust_network, min_component_size)

        # The number of nodes in robust network is monotonically increasing with `top_n`, so once a robust network is found
        # such that the number of nodes exceeds `max_size`, we do not need to iterate through the rest of the for loop and
        # can return the last valid robust network. 
        if robust_network.number_of_nodes() <= max_size: 
            robust_network_out = robust_network
        else: 
            return robust_network_out

    return robust_network_out

def filter_graph_by_component_size(nxgraph, min_size=5):
    """
    Removes any components that are less than `min_size`.

    Arguments:
        nxgraph (networkx.Graph): Network from randomization experiment
        min_size (int): Min size of components in `nxgraph`. Set to 2 to remove singletons only.

    Returns:
        networkx.Graph: Network with components less than specified size removed.
    """

    filtered_subgraph = nxgraph.copy()

    small_components = [nxgraph.subgraph(c).nodes() for c in nx.connected_components(nxgraph) if nxgraph.subgraph(c).number_of_nodes() < min_size]
    filtered_subgraph.remove_nodes_from(oi.flatten(small_components))

    return filtered_subgraph


In [31]:
def output_networkx_graph_as_interactive_html(nxgraph, attribute_metadata=dict(), output_dir=".", filename="graph.html"):
    """
    Arguments:
        nxgraph (networkx.Graph): any instance of networkx.Graph
        output_dir (str): the directory in which to output the file
        filename (str): the filename of the output file
    Returns:
        Path: the filepath which was outputted to
    """
    return axial.graph(nxgraph,
        title='OmicsIntegrator2 Results',
        scripts_mode="inline",
        data_mode="inline",
        output_dir=output_dir,
        filename=filename)


## Cell Specific Network Validation Functions

In [1]:
# int_df is interactome dataframe with cell_rank columns
# celltype: string (e.g. "Astrocyte_rank")
# network: your query network

# pilot approach: use the same interactome annotations (based on edges, taking min rank (less expresed)
def get_average_rank_score_from_interactome_dataframe(celltype, int_df, network):
    # get edge list
    edges_df = nx.to_pandas_edgelist(network)
    scores = 0
    for index, rows in edges_df.iterrows():
        protein1 = rows['source']
        protein2 = rows['target']
        scores_to_add = int_df[(int_df['protein1'] == protein1) & (int_df['protein2'] == protein2)][celltype]
        scores = scores + scores_to_add
    # average
    final_score = scores/len(robust_edges_df)
    
    
    

In [1]:
# networks: dictionary of networks
def jaccard_network(networks):
    pairwise_jaccard_df = pd.DataFrame(index = list(networks.keys()), columns = list(networks.keys()))
    for key1 in pairwise_jaccard_df.index:
        for key2 in pairwise_jaccard_df.index:
            network_1 = networks[key1]
            network_2 = networks[key2]
            L1 = list(network_1.edges)
            L2 = list(network_2.edges)
            J = jaccard(L1, L2)
            pairwise_jaccard_df.loc[key1, key2] = J
            pairwise_jaccard_df = pairwise_jaccard_df.astype('float')
    return pairwise_jaccard_df

In [2]:
# This function implements GSEA running ES score calculation
# Input:
# ranked_gene_list: a list of gene names ranked by its phenotype_corr_vals
# phenotype_corr_vals: a list of phenotype correlation values used to rank genes, 
#                         should be in the same order as ranked_gene_list 
# gene_set : user defined gene set (a list of gene names)
# p : P_hit exponent value

# ranked_genes_df : dataframe containing ranked genes and phenotype correlation values (i.e. logfoldchange here)

# Output:
# ES_vec: containes ES score for each gene, cumulative.
# max_Es : ES score (positive or negative) that has absolute maximum value
def GSEA(ranked_genes_df, gene_set, p):
    
    ranked_genes_df = ranked_genes_df.sort_values(by = 'logfoldchanges', ascending=False)
    phenotype_corr_vals = ranked_genes_df['logfoldchanges'].values
    ranked_gene_list = ranked_genes_df['names'].values
    
    
    # initialize ES score, ES_vec
    ES_score = 0
    ES_vec = []
    
    # define N, N_H, N_R
    N = len(ranked_gene_list)
    N_H = len(gene_set)
    
    N_R = 0 # initialize N_R
    for index, row in ranked_genes_df.iterrows():
        if row['names'] in gene_set:
            N_R += abs(row['logfoldchanges'])**p
    
    # walk down the ranked_gene_list
    for i in range(len(ranked_gene_list)):
        gene = ranked_gene_list[i]
        if gene in gene_set:
            # if gene is found in gene_set, update ES_score as follows
            P_hit = (abs(phenotype_corr_vals[i]))**p/N_R
            ES_score += P_hit
            ES_vec.append(ES_score)
        else:
            # if not found, update ES_score as follows
            P_miss = 1/(N-N_H)
            ES_score -= P_miss
            ES_vec.append(ES_score)
    
    # get maximum ES score (in both positive and negative)
    max_ES = np.max(np.abs(ES_vec)) # find absolute maximum
    for i in range(len(ES_vec)):
        if abs(ES_vec[i]) == max_ES:
            max_ES = ES_vec[i] # convert back to non-absolute value
    return ES_vec, max_ES




# Defining a function to use to permute labels
# input: 
# ranked_genes:  dataframe containing ranked genes and absolute SNR values
#                   to be permuted

# output:
# permuted_ranked_genes : data with gene labels permuted

def permute_labels(ranked_genes_df):
    
    permuted_ranked_genes = ranked_genes_df.copy()
    idx = np.random.permutation(len(ranked_genes_df))
    permuted_ranked_genes['names'] = ranked_genes_df.iloc[idx]['names'].values
    return permuted_ranked_genes
    
def permute_gene_sets(ranked_genes_df, gene_set):
    import random
    k = len(gene_set)
    gene_list = list(ranked_genes_df['names'].values)
    permuted_gene_set = random.sample(gene_list, k)
    return permuted_gene_set 
    
def get_null_enrichment_score(N, ranked_genes_df, gene_set, p):
    # define other inputs
    ranked_gene_list = ranked_genes_df['names'].values.tolist()
    # p = 1
    
    # initialize ES_null
    ES_null = []
    print('permuting gene sets')
    # for each permutation, update ES_null
    for j in range(N): # N = 1000
        # permute labels
        #permuted_ranked_genes_df = permute_labels(ranked_genes_df)
        
        # permute gene set 
        permuted_gene_set = permute_gene_sets(ranked_genes_df, gene_set)
        
        # run GSEA with updated list and append to ES_null[i]
        _, maxES_null = GSEA(ranked_genes_df, permuted_gene_set, p)
        ES_null.append(maxES_null)
    return ES_null

# ES_null = list of ES values from random permutation
# ES_true = query max ES to see whether it is as extreme as random distribution
def get_empirical_pval(ES_null, ES_true):
    if ES_true < 0: # if negative,
        # subset negative portion
        ES_null_subset= [x for x in ES_null if x < 0]
        # total number of permutations to compare
        N = len(ES_null_subset)
        # update nominal p-val, left tail 
        if N==0:
            pval = 0.0
        else:
            pval = len([x for x in ES_null_subset if x <= ES_true])/N
    else:
        # subset positive portion
        ES_null_subset= [x for x in ES_null if x >= 0]
        # total number of permutations to compare 
        N = len(ES_null_subset)
        if N==0:
            pval = 0.0
        else:
            # update nominal p-val, right tail
            pval =len([x for x in ES_null_subset if x >= ES_true])/N

    return pval
    


In [ ]:
pairwise_df_1 = pd.DataFrame(index = list(final_networks_1.keys()), columns = list(final_networks_1.keys()))
for key1 in pairwise_df_1.index:
    for key2 in pairwise_df_1.index:
        network_1 = final_networks_1[key1]
        network_2 = final_networks_1[key2]
        L1 = list(network_1.edges)
        L2 = list(network_2.edges)
        J = util.jaccard(L1, L2)
        pairwise_df_1.loc[key1, key2] = J
pairwise_df_1 = pairwise_df_1.astype('float')
pairwise_df_1

## Annotate 

In [36]:
def annotate_graph(clusters):
    anno_df = pd.read_csv('./effect_size_annotation.csv', index_col=0)
    # annotate GWA effect size, direction
    for index, row in anno_df.iterrows():
        for i in range(0,len(clusters)):
            network = clusters[i]
            score=round(row['GWA Effect Size'], 2)
            direction = row['Direction']
            nx.set_node_attributes(network, {index: {'GWA Effect Size': score, 'Direction': str(direction)}})
    for i in range(0, len(clusters)):
        for node in clusters[i].nodes():
            if node in anno_df.index:
                pass
            else:
                nx.set_node_attributes(clusters[i], {node: {'Direction': 'NA'}})     
                
    # sort annotation for consistent display
    for i in range(0, len(clusters)):
        nodes_sorted2 = sorted(clusters[i].nodes(data=True), key = lambda x: (x[1]['source'], x[-1]['Direction']))
        edges2 = sorted(clusters[i].edges(data=True))
        clusters[i] = nx.Graph()
        clusters[i].add_nodes_from(nodes_sorted2)
        clusters[i].add_edges_from(edges2)
        for node, d in clusters[i].nodes(data=True):
            if node in anno_df.index:
                pass
            else:
                d.pop('Direction', None)

## Test

In [8]:
def hello():
    print('test')

In [9]:
hello()

test


In [1]:
def hello2():
    print('test')

## Network Comparisons and Validation for cell specific signal

In [3]:
# compare edge cell expr rank percentiles
# networks: dictionary of cell specific network, key-indexed by cell type names - pre-selected for hyperparam
def get_edge_scores_df(networks, int_df, mode='forest'):
    rank_scores_df = pd.DataFrame(index = list(networks.keys()), columns = list(int_df.columns)[3:])
    for key1 in rank_scores_df.index:
        network = networks[key1]
    
        edge_df = nx.to_pandas_edgelist(network)
        # if forest mode, then only take PCSF selected edges.
        if mode == 'forest':
            edge_df = edge_df[edge_df.in_solution == True]
        if key1 == 'Naive':
            df_result1 = pd.merge(edge_df.copy(), int_df, left_on = ['source','target'], right_on = ['protein1','protein2'])
            df_result2= pd.merge(edge_df.copy(), int_df, left_on = ['source','target'], right_on = ['protein2','protein1'])
            df_merged= df_result1.append(df_result2) # contains score info
    
        else:
            df_merged = edge_df.copy()
    
        for key2 in rank_scores_df.columns:    
            celltype = key2
            rank_score = np.sum(df_merged[celltype])/len(df_merged)
        
            rank_scores_df.loc[key1, key2] = rank_score
    rank_scores_df = rank_scores_df.astype('float')
    return rank_scores_df

In [5]:
# compare edge cell expr rank percentiles
# networks: dictionary of cell specific network, key-indexed by cell type names - pre-selected for hyperparam
# expr_df = adult_cortex_df or juv_cortex_df

def get_node_scores_df(networks, int_df, expr_df, mode='Terminals'):
    rank_scores_df = pd.DataFrame(index = list(networks.keys()), columns = list(int_df.columns)[3:])
    for key1 in rank_scores_df.index:
        network = networks[key1]
    
        node_df = oi.get_networkx_graph_as_dataframe_of_nodes(network)

        # if forest mode, then only take PCSF selected edges.
        if mode == 'Terminals':
            node_df = node_df[node_df.terminal == True]
        elif mode == 'Non-terminals':
            node_df = node_df[node_df.terminal == False]
        else:
            print('proceeding with all nodes')
            
        inds = list(set(node_df.index.tolist()).intersection(set(expr_df.index.tolist())))
    
        for key2 in rank_scores_df.columns:    
            celltype = key2 # i.e. Astrocyte_rank
            gene_ranks = list(expr_df.loc[inds, celltype].values)
        
            rank_scores_df.loc[key1, key2] = sum(gene_ranks)/len(node_df)
    rank_scores_df = rank_scores_df.astype('float')
    return rank_scores_df


In [6]:
# if mode = 'forest' then only subset edges that are selected by PCSF
# calculate pairwise jaccard index of networks
def pairwise_jaccard_index_from_networks(networks, mode = 'augmented_forest'):
    pairwise_df = pd.DataFrame(index = list(networks.keys()), columns = list(networks.keys()))
    for key1 in pairwise_df.index:
        for key2 in pairwise_df.index:
            network_1 = networks[key1]
            network_2 = networks[key2]
            L1 = list(network_1.edges)
            L2 = list(network_2.edges)
            if mode == 'forest':
                edge_df1 = nx.to_pandas_edgelist(network_1)
                edge_df2 = nx.to_pandas_edgelist(network_2)
                G1 = nx.from_pandas_edgelist(edge_df1[edge_df1.in_solution==True], source='source',target='target')
                G2 = nx.from_pandas_edgelist(edge_df2[edge_df2.in_solution==True], source='source',target='target')
                L1 = list(G1.edges)
                L2 = list(G2.edges)
            J = jaccard(L1, L2)
            pairwise_df.loc[key1, key2] = J
    pairwise_df = pairwise_df.astype('float')
    return pairwise_df

In [7]:
# if mode = 'forest' then only subset edges that are selected by PCSF
# calculate pairwise jaccard index of networks
def pairwise_jaccard_index_from_network_nodes(networks, mode = 'Terminal'):
    
    pairwise_df = pd.DataFrame(index = list(networks.keys()), columns = list(networks.keys()))
    for key1 in pairwise_df.index:
        for key2 in pairwise_df.index:
            network_1 = networks[key1]
            network_2 = networks[key2]
            node_df1 = oi.get_networkx_graph_as_dataframe_of_nodes(network_1)
            node_df2 = oi.get_networkx_graph_as_dataframe_of_nodes(network_2)
            if mode == 'Terminal':
                L1 = list(node_df1[node_df1['terminal'] == True].index)
                L2 = list(node_df2[node_df2['terminal'] == True].index)
            elif mode == 'Non-terminal':
                L1 = list(node_df1[node_df1['terminal'] == False].index)
                L2 = list(node_df2[node_df2['terminal'] == False].index)
            elif mode == 'All':
                L1 = list(node_df1.index)
                L2 = list(node_df2.index)
                            
                
            intersection = len(list(set(L1).intersection(set(L2))))
            union = (len(L1) + len(L2)) - intersection
                               
            J = float(intersection) / union
            pairwise_df.loc[key1, key2] = J
    pairwise_df = pairwise_df.astype('float')
    return pairwise_df
